In [ ]:
!pip install -U textblob
!python -m textblob.download_corpora
!pip install bs4

In [ ]:
import os
import pandas as pd
import re
import string
import nltk

from bs4 import BeautifulSoup as bs
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
def load_files():

  path = '/content/drive/MyDrive/졸프/youtuberdata/ConservativeCaptionList'

  files = []

  for r, d, f in os.walk(path):
      for file in f:
          if '.txt' and 'TO' in str(file):
            
            files.append(os.path.join(r, file))
  
  return files

In [ ]:
paths = load_files()

In [ ]:
paths

In [ ]:
raw_caption_list = []
clean_caption_list = []
polarity_list = []
subjectivity_list = []
sentiment_list = []
emotion_list = []

In [ ]:
for path in paths:
  with open(path) as c:
    lines = c.readlines()
    for line in lines:
      raw_caption_list.append(line)

In [ ]:
len(raw_caption_list)

In [ ]:
def preprocessing(review, remove_stopwords = False ): 
    review_text = bs(review, "html5lib").get_text()	

    review_text = re.sub("[^a-zA-Z]", " ", review_text)

    words = review_text.lower().split()

    if remove_stopwords: 
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        clean_review = ' '.join(words)
    else:
        clean_review = ' '.join(words)

    return clean_review

In [ ]:
def cleaning_caption(data):
  clean_captions = []
  for caption in data:
      clean_captions.append(preprocessing(caption,remove_stopwords=True))
  return clean_captions

In [ ]:
clean_caption_list = cleaning_caption(raw_caption_list)

In [ ]:
def sentiment_analysis(clean_caption):
  result=TextBlob(clean_caption)
  polarity_list.append(result.polarity)
  subjectivity_list.append(result.subjectivity)


In [ ]:
def process_sentiment():
  for sentiment in polarity_list:
    if sentiment>= 0.5 and sentiment<=1.0:
	    sentiment_list.append("strongPositive")
    elif sentiment>0.0 and sentiment<0.5:
      sentiment_list.append("positive")
    elif sentiment == 0:
      sentiment_list.append("neutral")
    elif sentiment <0 and sentiment>= -0.5:
      sentiment_list.append("negative")
    elif sentiment < -0.5:
      sentiment_list.append("strongNegative")
      

emotion분석

In [ ]:
emotion_file = pd.read_csv('/content/drive/MyDrive/졸프/NRC-Emotion-Lexicon-clean.csv',encoding = 'utf-8-sig')
emotion_file.info

In [ ]:
def flat_list(array): 
    a=[]
    for i in array:
        if type(i) == type(list()):
            a+=(flat_list(i))       
        else:
            a.append(i)
    return a

In [ ]:
def extract_emotion_list(text,file):

  lower_case = text.lower()

  cleaned_text = lower_case.translate(str.maketrans('', '', string.punctuation))

  tokenized_words = word_tokenize(cleaned_text, "english")

# Lemmatization - From plural to single + Base form of a word (example better-> good)
  lemma_words = []
  for word in tokenized_words:
      word = WordNetLemmatizer().lemmatize(word)
      lemma_words.append(word)

  emotion_lists = []

  for j in range(len(file['wordlist'])):
      word = file['wordlist'][j]

      if word in lemma_words:
          emotion_lists.append(file['emotions'][j].split(':'))
  emotion_lists = flat_list(emotion_lists)
  return emotion_lists



In [ ]:
for i in range(len(clean_caption_list)):
  if i%1000 == 0:
    print(i)
  sentiment_analysis(clean_caption_list[i])
  emotion_list.append(extract_emotion_list(clean_caption_list[i],emotion_file))
process_sentiment()

In [ ]:
df = pd.DataFrame({'raw_comment':raw_caption_list, 'clean_comment':clean_caption_list,
                   'polarity':polarity_list,'subjectivity':subjectivity_list,
                   'sentiment':sentiment_list,'emotion_list':emotion_list})

In [ ]:
df.to_csv("/content/drive/MyDrive/졸프/감성분석result/Conservative_Result/T0_Conservative_Caption_result.csv",encoding = 'utf-8-sig')

In [ ]:
  df